## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-09-18-49-04 +0000,nypost,Ferrari evokes spirit of 1980s with new 849 Te...,https://nypost.com/2025/09/09/business/ferrari...
1,2025-09-09-18-48-18 +0000,nyt,Live Updates: Nepali Troops Move to Restore Or...,https://www.nytimes.com/live/2025/09/09/world/...
2,2025-09-09-18-48-17 +0000,nyt,Nepal’s Prime Minister Resigns as Protesters S...,https://www.nytimes.com/live/2025/09/09/world/...
3,2025-09-09-18-47-22 +0000,wapo,Israel Defense Forces chief of staff Eyal Zami...,https://www.washingtonpost.com
4,2025-09-09-18-47-22 +0000,wapo,Live updates: Israeli strike targeting Hamas l...,https://www.washingtonpost.com/world/2025/09/0...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2336/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
44,trump,38
3,new,30
15,nepal,15
399,media,14
196,epstein,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
38,2025-09-09-17-44-10 +0000,nypost,White House fumes over Epstein birthday book s...,https://nypost.com/2025/09/09/us-news/wh-fumes...,94
197,2025-09-09-09-04-25 +0000,cbc,Nepal's PM resigns after violent protests over...,https://www.cbc.ca/news/world/nepal-protests-g...,90
218,2025-09-09-07-10-42 +0000,nypost,Bill Clinton letter in Epstein ‘birthday book’...,https://nypost.com/2025/09/09/us-news/bill-cli...,89
192,2025-09-09-09-30-00 +0000,wsj,When a federal appeals court recently rejected...,https://www.wsj.com/politics/policy/cases-on-t...,89
17,2025-09-09-18-28-45 +0000,nypost,French President Emmanuel Macron names loyalis...,https://nypost.com/2025/09/09/world-news/frenc...,86


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
38,94,2025-09-09-17-44-10 +0000,nypost,White House fumes over Epstein birthday book s...,https://nypost.com/2025/09/09/us-news/wh-fumes...
197,90,2025-09-09-09-04-25 +0000,cbc,Nepal's PM resigns after violent protests over...,https://www.cbc.ca/news/world/nepal-protests-g...
150,84,2025-09-09-12-07-44 +0000,nypost,Zohran Mamdani would still beat Andrew Cuomo i...,https://nypost.com/2025/09/09/us-news/zohran-m...
4,61,2025-09-09-18-47-22 +0000,wapo,Live updates: Israeli strike targeting Hamas l...,https://www.washingtonpost.com/world/2025/09/0...
192,51,2025-09-09-09-30-00 +0000,wsj,When a federal appeals court recently rejected...,https://www.wsj.com/politics/policy/cases-on-t...
314,48,2025-09-08-20-48-15 +0000,cbc,Freight train slams into a bus outside Mexico ...,https://www.cbc.ca/news/world/train-bus-collis...
279,44,2025-09-08-22-53-00 +0000,nypost,Lachlan Murdoch takes control of media empire ...,https://nypost.com/2025/09/08/media/lachlan-mu...
12,44,2025-09-09-18-34-15 +0000,nyt,Macron Picks Sébastien Lecornu as France’s Nex...,https://www.nytimes.com/2025/09/09/world/europ...
110,42,2025-09-09-14-49-29 +0000,nypost,Greta Thunberg’s Gaza flotilla was not hit by ...,https://nypost.com/2025/09/09/world-news/greta...
172,38,2025-09-09-10-23-48 +0000,nypost,Eerie bodycam footage shows the Amish mother a...,https://nypost.com/2025/09/09/us-news/ohio-ami...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
